In [1]:
import pandas as pd
import sqlite3

# Objectives

- Use SQL subqueries to nest queries
- Identify common SQL dialects and tools
- Query data from web databases

![yo dawg sql meme](images/yodawg.jpeg)

# SQL Subqueries

Like you might nest one function within another in Python, you can nest queries in SQL. We can use a **subquery** within another query to succinctly implement queries that have multiple query steps.

In [2]:
conn = sqlite3.connect('data/flights.db')

In [3]:
connection = sqlite3.Connection('data/flights.db')

In [4]:
type(conn), type(connection)

(sqlite3.Connection, sqlite3.Connection)

## Subqueries in `FROM`

You can use a subquery in the `FROM` clause - this is useful, for example, if you want to apply multiple aggregation functions.

Let say we want to get the average of the number of routes departing from all airports. First we'd need to get the total number of routes departing from all airports, then take the average.

In [5]:
pd.read_sql('''

SELECT source AS departing_airport, COUNT() AS num_departures 
FROM routes
GROUP BY source

''', conn)

,departing_airport,num_departures
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


We can use this query as a subquery, and take the average of the new `number_of_departures` column.

In [7]:
pd.read_sql('''

SELECT AVG(num_departures)
FROM (SELECT source AS departing_airport, COUNT() AS num_departures 
        FROM routes
        GROUP BY source)

''', conn)

,AVG(num_departures)
0,19.848343


## Note: Subqueries are Like New Tables!

If you squint, you'll notice that the subquery is taking the place of where we might put a table!

For example, checkout the SQL we wrote in our first subquery example:

```sql
SELECT 
    AVG(number_of_departures)
FROM (
    SELECT 
        source AS depart_airport
        ,COUNT() AS number_of_departures
    FROM
        routes
    GROUP BY
        source
)
```

We could imagine that some new table that returned by the subquery existed (let's call it `airport_departures`) and be placed in place of the subquery:

```sql
SELECT 
    AVG(number_of_departures)
FROM (
    airport_departures -- Replacing subquery with this hypothetical table
) 
```

You can actually use syntax close to this with **Common Table Expressions (CTEs)** found in the [Level Up section](#Level-Up:-Common-Table-Expressions) below.

## Subqueries in `WHERE`

You can use a subquery in the `WHERE` clause - this is useful, for example, if you want to filter a query based on results from another query.

Let's say that we want to get a table with all of the departures and destinations for the flight routes, but I only want to include flights departing from the five countries with the most airports.

To do this, we'd first need to identify the five countries that have the most airports. 

In [14]:
pd.read_sql('''
SELECT country, COUNT() AS num_airports
FROM airports
GROUP BY country
ORDER BY COUNT() DESC
LIMIT 5


''', conn)

,country,num_airports
0,United States,1697
1,Canada,435
2,Germany,321
3,Australia,263
4,Russia,249


I could enter these results into a new query of the routes table to get the data I want.

In [15]:
pd.read_sql('''
SELECT *
FROM routes
ORDER BY source_id 

''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,17312,CG,1308,GKA,1,HGU,3,None,0,DH8 DHT
1,17313,CG,1308,GKA,1,LAE,4,None,0,DH8
2,17314,CG,1308,GKA,1,MAG,2,None,0,DH8
3,17315,CG,1308,GKA,1,POM,5,None,0,DH8
4,46428,PX,328,GKA,1,POM,5,None,0,DH4 DH8 DH3
...,...,...,...,...,...,...,...,...,...,...
67658,66304,Z3,1266,KWF,\N,KTN,3808,None,0,DHP
67659,66352,Z8,603,ORU,\N,CBB,2759,None,0,SWM
67660,66353,Z8,603,ORU,\N,SRZ,4247,None,0,SWM
67661,66822,ZH,4611,DQA,\N,PEK,3364,Y,0,737


In [16]:
pd.read_sql('''
SELECT *
FROM airports

''', conn)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8102,8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8,A,America/Vancouver
8103,8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8,A,America/Vancouver
8104,8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8,A,America/Vancouver
8105,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


In [18]:
pd.read_sql('''

SELECT r.source AS departing_airport, r.dest AS destination_airport, a.country AS departing_country
FROM routes as r
    LEFT JOIN airports AS a
        ON r.source_id = a.id
WHERE a.country IN ('United States', 'Canada', 'Germany', 'Australia', 'Russia')
-- comment: we entered these 5 countries we found before by hand, but it's hard coded. What if these change? Use a subquery!
ORDER BY departing_country


''', conn)

,departing_airport,destination_airport,departing_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


This approach works but has a few limitations:

- We have to manually enter the countries to filter them
- The list of countries won't update with our data, so we'd have to monitor and manually change them in the future
- We have to look at two separate queries to understand what our code is supposed to do
- We have to run two separate queries, which might take longer than one combined query

A better solution uses a subquery to get the list of 5 countries and feed it into our WHERE clause.

In [ ]:
pd.read_sql('''

SELECT r.source AS departing_airport, r.dest AS destination, a.country AS departing_country
FROM routes AS r
    LEFT JOIN airports AS a
        ON r.source_id = a.id
WHERE a.country IN (SELECT country
                    FROM airports
                    GROUP BY country
                    ORDER BY COUNT() DESC
                    LIMIT 5)
                    -- Here's our subquery
ORDER BY departing_country

''', conn)

## Level Up: Common Table Expressions

Common Table Expressions (CTEs) are a more readable way to implement subqueries, using `WITH` and `AS`.

In [19]:
pd.read_sql('''
WITH top_5_countries AS (SELECT country
                    FROM airports
                    GROUP BY country
                    ORDER BY COUNT() DESC
                    LIMIT 5)

SELECT 
    rt.source AS depart_airport
    ,rt.dest AS destination_airport
    ,ap.country AS depart_country
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    ap.country IN top_5_countries
ORDER BY 
    depart_country
''', conn)

,depart_airport,destination_airport,depart_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


### Exercise

Create a table listing all airlines that serve the three airports with the most outbound routes.

In [20]:
pd.read_sql('''
SELECT *
FROM airports

''', conn)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8102,8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8,A,America/Vancouver
8103,8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8,A,America/Vancouver
8104,8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8,A,America/Vancouver
8105,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


In [21]:
pd.read_sql('''
SELECT *
FROM routes

''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


In [22]:
pd.read_sql('''
SELECT *
FROM airlines

''', conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
...,...,...,...,...,...,...,...,...,...
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y


In [28]:
## Your work here
## in this case we want inner join b/c otherwise all the nulls get counted up
## and end up at the top of the table??
## we established the shared keys by looking at the tables the other day
pd.read_sql("""

SELECT ap.id
FROM airports AS ap
    JOIN routes AS r 
        ON r.source_id = ap.id 
GROUP BY ap.id
ORDER BY COUNT() DESC
LIMIT 3

""", conn)

,id
0,3682
1,3830
2,3364


In [29]:
pd.read_sql("""

WITH top_3 AS (SELECT ap.id
FROM airports AS ap
    JOIN routes AS r 
        ON r.source_id = ap.id 
GROUP BY ap.id
ORDER BY COUNT() DESC
LIMIT 3)

SELECT DISTINCT r.airline
FROM routes AS r
    JOIN airports AS ap
        ON r.source_id = ap.id
WHERE r.source_id IN top_3

""", conn)

# Need the select distinct to just get one entry for each airline

,airline
0,3E
1,3M
2,3U
3,5J
4,8L
...,...
97,WN
98,WS
99,Y4
100,Y7


<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql
WITH top_3_airports AS (
    SELECT 
        airports.id 
    FROM 
        airports 
        LEFT JOIN routes
            ON routes.source_id = airports.id
    GROUP BY
        airports.id
    ORDER BY 
        COUNT() DESC
    LIMIT 3
) 

SELECT DISTINCT
    rt.airline
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    rt.source_id IN top_3_airports
```
</details>

# SQL Versions

There is no one version of SQL - there are many versions out there! What you're learning about SQL with SQLite will apply to all of them. Just keep in mind when you apply for jobs that you may see any of these listed in any given job posting, and they are all just different versions of what you know.

## SQL Dialects

As with dialects of spoken languages, SQL dialects have many commonalities but some differences in syntax and functionality.  Here are a few of the major players:

- SQLite (we've already seen this!)
- PostgreSQL (free and open-source!)
- Oracle SQL
- MySQL (half open-souce, half Oracle)
- Microsoft SQL Server
- Transact-SQL (extends MS SQL)

## SQLite Pros & Cons

We use SQLite in this course, but it has some limitations.

### Pros

- Easy to set up
- Easy to share database files
- Uses little memory

### Cons

- Limited functionality for managing users and access permissions
- Not "thread safe": two edits at the same time can mess up your data

## Extra Resources: SQL Versions

[What Is a SQL Dialect, and Which one Should You Learn?](https://learnsql.com/blog/what-sql-dialect-to-learn/)

[SQLite vs MySQL vs PostgreSQL](https://www.digitalocean.com/community/tutorials/sqlite-vs-mysql-vs-postgresql-a-comparison-of-relational-database-management-systems)

[SQL Dialect Reference](https://en.wikibooks.org/wiki/SQL_Dialects_Reference)

# Web Databases: data.world

For the rest of this lesson, we'll be exploring databases in [data.world](https://data.world/), a web database that we can query using SQL in our browser. For reference, you can see the instructions for creating a new project here: [Getting Started Working with Data at data.world](https://docs.data.world/en/64499-94893-Work-with-data--Projects.html)

## Step 1: Create a data.world account

You will need to enter and verify your email address.

## Step 2: Create a project using this [Austin AirBnB](https://data.world/jonloyens/inside-airbnb-austin) dataset

Navigate to [this page](https://data.world/jonloyens/inside-airbnb-austin) and use the button at the top right of the page to create a new project using the data.

![](images/data_world_austin_airbnb_new_proj.png)

## Step 3: Create a SQL query

In your project, use the "+ Add" button to add a SQL query.

![](images/data_world_add_sql_query.png)

## Step 4: Run a simple SQL query

Try entering "SELECT * FROM listings" and clicking the "Run Query" button in the top right.

## Step 5: Practice using SQL to explore the data

Below are some exercises to practice your SQL skills and help explore the data. You will need to explore the table schemas to complete these exercises. 

You may find it helpful to click on the corresponding .csv files to inspect the data, or look at the columns in each table in the right sidebar.

### Exercise 1: Create a table showing the number of listings in each neighborhood

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>
    
```sql
    SELECT neighbourhood, COUNT(*)
    FROM listings
    GROUP BY neighbourhood
```   
</details>

### Exercise 2: Create a table showing the 20 listings with the most reviews

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql
    SELECT *
    FROM listings
    ORDER BY number_of_reviews DESC
    LIMIT 20
```
</details>

### Exercise 3: Create a table showing all of the reviews for listings that are "Bed & Breakfast" property types.

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql    
    SELECT *
    FROM reviews
    INNER JOIN listings
        ON listings.id = reviews.listing_id
    WHERE listings.property_type = "Bed & Breakfast"
```
</details>

### Exercise 4: Run your own query using a subquery or CTE.

Note that the syntax and functionality for subqueries and CTEs in data.world are more limited than SQLite, so try creating simple ones.